In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleaned-data/cleaned_data.csv


In [4]:
import torch, os
import pandas as pd
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast
from transformers import RobertaForSequenceClassification, AutoTokenizer
from torch.utils.data import Dataset

2024-06-21 12:35:40.549380: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-21 12:35:40.549481: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-21 12:35:40.793916: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [6]:
PATH = '/kaggle/input/cleaned-data/cleaned_data.csv'
df_org= pd.read_csv(PATH)
df_org

,Label,Name
0,ghế_văn_phòng,ghế hq
1,thước_dây,thước dây làm bằng sợi thủy tinh total
2,cưa_tay,cưa gỗ cầm tay cán lớn asaki ak
3,"váy,_đầm",đầm ren cổ v tay ngắn cao cấp
4,"dụng_cụ_đo,_kiểm_tra_khác",máy đo độ đồng tâm hann yan
...,...,...
14971,máy_làm_lạnh_nước,máy làm lạnh nước orion
14972,bàn_ghế_trang_điểm,bàn phấn mdf
14973,bàn_phím,bàn phím rapoo
14974,sàn_gỗ,sàn gỗ morser ly


In [7]:
labels = df_org['Label'].unique().tolist()
labels = [s.strip() for s in labels ]
labels[:5]

['ghế_văn_phòng',
 'thước_dây',
 'cưa_tay',
 'váy,_đầm',
 'dụng_cụ_đo,_kiểm_tra_khác']

In [8]:
NUM_LABELS= len(labels)

id2label={id:label for id,label in enumerate(labels)}

label2id={label:id for id,label in enumerate(labels)}

In [9]:
#label2id
#id2label

In [10]:
df_org["labels"]=df_org.Label.map(lambda x: label2id[x.strip()])

In [11]:
df_org.head()

,Label,Name,labels
0,ghế_văn_phòng,ghế hq,0
1,thước_dây,thước dây làm bằng sợi thủy tinh total,1
2,cưa_tay,cưa gỗ cầm tay cán lớn asaki ak,2
3,"váy,_đầm",đầm ren cổ v tay ngắn cao cấp,3
4,"dụng_cụ_đo,_kiểm_tra_khác",máy đo độ đồng tâm hann yan,4


In [12]:
# Giả sử df_org đã được định nghĩa
SIZE = df_org.shape[0]

# Tính toán số lượng mẫu cho từng tập
train_size = int(0.7 * SIZE)
val_size = int(0.15 * SIZE)
test_size = SIZE - train_size - val_size  # Cái này sẽ đảm bảo đủ 100%

# Chia dữ liệu
train_texts = list(df_org.Name[:train_size])
val_texts = list(df_org.Name[train_size:train_size + val_size])
test_texts = list(df_org.Name[train_size + val_size:])

train_labels = list(df_org.labels[:train_size])
val_labels = list(df_org.labels[train_size:train_size + val_size])
test_labels = list(df_org.labels[train_size + val_size:])

In [13]:
len(train_texts), len(val_texts), len(test_texts)

(10483, 2246, 2247)

In [14]:
# Kiểm tra và in loại dữ liệu của các phần tử trong danh sách
print(type(train_texts[0]))  # Phải là <class 'str'>
print(type(val_texts[0]))    # Phải là <class 'str'>
print(type(test_texts[0]))   # Phải là <class 'str'>

<class 'str'>
<class 'str'>
<class 'str'>


In [15]:
# Tải mô hình và tokenizer
model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", num_labels=NUM_LABELS, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False, max_length=30)
# Chuyển model sang thiết bị (CPU/GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at wonrax/phobert-base-vietnamese-sentiment and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([1971, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([1971]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [16]:
# Kiểm tra các phần tử không phải chuỗi
non_str_elements = [text for text in train_texts if not isinstance(text, str)]
print("Các phần tử không phải chuỗi trong train_texts:", non_str_elements)

Các phần tử không phải chuỗi trong train_texts: [nan, nan, nan]


In [17]:
# Hàm để lọc bỏ các giá trị NaN
def filter_nan(texts, labels):
    filtered_texts = []
    filtered_labels = []
    for text, label in zip(texts, labels):
        if pd.notna(text):
            filtered_texts.append(text)
            filtered_labels.append(label)
    return filtered_texts, filtered_labels

# Lọc bỏ các giá trị NaN
train_texts, train_labels = filter_nan(train_texts, train_labels)
val_texts, val_labels = filter_nan(val_texts, val_labels)
test_texts, test_labels = filter_nan(test_texts, test_labels)

In [18]:
len(test_texts), len(test_labels)

(2247, 2247)

In [19]:
# Check the data type of each element after filtering
assert all(isinstance(text, str) for text in train_texts), "Train texts contain elements that are not strings"
assert all(isinstance(text, str) for text in val_texts), "Val texts contain elements that are not strings"
assert all(isinstance(text, str) for text in test_texts), "Test texts contain elements that are not strings"

In [20]:
# Tokenizer
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Length check
print(len(train_texts), len(val_texts), len(test_texts))


10480 2246 2247


In [22]:
class DataLoader(Dataset):
    """
    Custom Dataset class for handling tokenized text data and corresponding labels.
    Inherits from torch.utils.data.Dataset.
    """
    def __init__(self, encodings, labels):
        """
        Initializes the DataLoader class with encodings and labels.

        Args:
            encodings (dict): A dictionary containing tokenized input text data
                              (e.g., 'input_ids', 'token_type_ids', 'attention_mask').
            labels (list): A list of integer labels for the input text data.
        """
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        """
        Returns a dictionary containing tokenized data and the corresponding label for a given index.

        Args:
            idx (int): The index of the data item to retrieve.

        Returns:
            item (dict): A dictionary containing the tokenized data and the corresponding label.
        """
        # Retrieve tokenized data for the given index
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Add the label for the given index to the item dictionary
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        """
        Returns the number of data items in the dataset.

        Returns:
            (int): The number of data items in the dataset.
        """
        return len(self.labels)

In [23]:
train_dataloader = DataLoader(train_encodings, train_labels)

val_dataloader = DataLoader(val_encodings, val_labels)

test_dataset = DataLoader(test_encodings, test_labels)

In [24]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    # Extract true labels from the input object
    labels = pred.label_ids
    
    # Obtain predicted class labels by finding the column index with the maximum probability
    preds = pred.predictions.argmax(-1)
    
    # Compute macro precision, recall, and F1 score using sklearn's precision_recall_fscore_support function
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro', zero_division=0)
    
    # Calculate the accuracy score using sklearn's accuracy_score function
    acc = accuracy_score(labels, preds)
    
    # Return the computed metrics as a dictionary
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

In [25]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning

warnings.filterwarnings("ignore", message="Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.")
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

In [26]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir='./pho-bert-classification', 
    do_train=True,
    do_eval=True,
    # The number of epochs, defaults to 3.0 
    num_train_epochs=100,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=32,
    # Number of steps used for a linear warmup
    warmup_steps=100,                
    weight_decay=0.01,
    logging_strategy='steps',
    # TensorBoard log directory                 
    logging_dir='./multi-class-logs',            
    logging_steps=500,  
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps", 
    save_steps=1000,  
    save_total_limit=2,  
    fp16=True,
    load_best_model_at_end=True
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [27]:
trainer = Trainer(
    # the pre-trained model that will be fine-tuned 
    model=model,
     # training arguments that we defined above                        
    args=training_args,                 
    train_dataset=train_dataloader,         
    eval_dataset=val_dataloader,            
    compute_metrics= compute_metrics
)

In [28]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
500,6.624400,5.730442,0.156723,0.011091,0.009311,0.022810
1000,5.017100,4.512115,0.328584,0.065537,0.057773,0.096725
1500,3.816100,3.689478,0.459484,0.148562,0.136974,0.192087
2000,2.999000,3.226604,0.512912,0.196123,0.184316,0.245165
2500,2.341000,2.872236,0.568121,0.251142,0.237330,0.305103
3000,1.885200,2.639239,0.605521,0.289429,0.274837,0.341353
3500,1.494000,2.512645,0.621995,0.321270,0.309403,0.366959
4000,1.191100,2.432720,0.637578,0.351082,0.343410,0.394595
4500,0.957800,2.377387,0.650490,0.365188,0.358593,0.405119
5000,0.774300,2.355080,0.653606,0.380818,0.375980,0.419496


TrainOutput(global_step=32800, training_loss=0.47234468554578174, metrics={'train_runtime': 9481.5561, 'train_samples_per_second': 110.53, 'train_steps_per_second': 3.459, 'total_flos': 2.0826906460704e+16, 'train_loss': 0.47234468554578174, 'epoch': 100.0})

In [39]:
q=[trainer.evaluate(eval_dataset=df_org) for df_org in [train_dataloader, val_dataloader, test_dataset]]

pd.DataFrame(q, index=["train","val","test"]).iloc[:,:5]

,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.210958,0.982156,0.929723,0.926232,0.937418
val,2.319848,0.679875,0.418374,0.416463,0.448993
test,2.190387,0.684913,0.440927,0.441439,0.471791


In [30]:
def predict(text):
    """
    Predicts the class label for a given input text

    Args:
        text (str): The input text for which the class label needs to be predicted.

    Returns:
        probs (torch.Tensor): Class probabilities for the input text.
        pred_label_idx (torch.Tensor): The index of the predicted class label.
        pred_label (str): The predicted class label.
    """
    # Tokenize the input text and move tensors to the GPU if available
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")

    # Get model output (logits)
    outputs = model(**inputs)

    probs = outputs[0].softmax(1)
    """ Explanation outputs: The BERT model returns a tuple containing the output logits (and possibly other elements depending on the model configuration). In this case, the output logits are the first element in the tuple, which is why we access it using outputs[0].

    outputs[0]: This is a tensor containing the raw output logits for each class. The shape of the tensor is (batch_size, num_classes) where batch_size is the number of input samples (in this case, 1, as we are predicting for a single input text) and num_classes is the number of target classes.

    softmax(1): The softmax function is applied along dimension 1 (the class dimension) to convert the raw logits into class probabilities. Softmax normalizes the logits so that they sum to 1, making them interpretable as probabilities. """

    # Get the index of the class with the highest probability
    # argmax() finds the index of the maximum value in the tensor along a specified dimension.
    # By default, if no dimension is specified, it returns the index of the maximum value in the flattened tensor.
    pred_label_idx = probs.argmax()

    # Now map the predicted class index to the actual class label 
    # Since pred_label_idx is a tensor containing a single value (the predicted class index), 
    # the .item() method is used to extract the value as a scalar
    pred_label = model.config.id2label[pred_label_idx.item()]

    return probs, pred_label_idx, pred_label

In [31]:
# Test with a an example text in Turkish
text = "ghế hq"
# "Machine Learning itself is moving towards more and more automated"
predict(text)[2]

'ghế_văn_phòng'

## Save model for inference

In [32]:
model_path = "pho-bert-classification"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

('pho-bert-classification/tokenizer_config.json',
 'pho-bert-classification/special_tokens_map.json',
 'pho-bert-classification/vocab.txt',
 'pho-bert-classification/bpe.codes',
 'pho-bert-classification/added_tokens.json')

## Re-Load saved model for inference

In [33]:
model_path = "pho-bert-classification"

In [34]:
# Load model and tokenizer
model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", num_labels=NUM_LABELS, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False, max_length=30)
# Switch model to device (CPU/GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at wonrax/phobert-base-vietnamese-sentiment and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([1971, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([1971]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [35]:
model = RobertaForSequenceClassification.from_pretrained(model_path)
tokenizer= AutoTokenizer.from_pretrained(model_path)
nlp= pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [55]:
nlp("máy lọc") 

[{'label': 'máy_lọc_nước', 'score': 0.692017138004303}]